In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time

### Задание 1

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>


#### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [6]:
KEYWORDS = ['python', 'парсинг']

In [7]:
KEYWORDS = [word.lower() for word in KEYWORDS]

habr_posts = pd.DataFrame()

habr_posts_req = requests.get('https://habr.com/ru/all/')
time.sleep(0.3)

soup = BeautifulSoup(habr_posts_req.text, 'html.parser')

posts = soup.find_all('article', class_='post')

for post in posts:
    post_link = post.find('a', class_='post__title_link').get('href')
    habr_post_req = requests.get(post_link)
    time.sleep(0.3)
    full_post = BeautifulSoup(habr_post_req.text, 'html.parser')
    for word in full_post.find('div', class_='post__text').text.split():
        if word.lower() in KEYWORDS:
            post_date = full_post.find('span', class_='post__time').text
            if 'сегодня' in post_date:
                post_date = datetime.datetime.now().strftime('%d.%m.%Y')
            elif 'вчера' in post_date:
                post_date = (datetime.datetime.now()-datetime.timedelta(days=1)).strftime('%d.%m.%Y')
            post_title = full_post.find('span', class_='post__title-text').text
            post_text = full_post.find('div', class_='post__text').text
            row = {'time': post_date, 'title': post_title, 'link': post_link, 'text': post_text}
            habr_posts = pd.concat([habr_posts, pd.DataFrame([row])])          
            break
        
habr_posts.head()

,time,title,link,text
0,03.12.2020,Cloud4Y запускает маркетплейс технологических ...,https://habr.com/ru/company/cloud4y/blog/531080/,Мы запускаем новый проект: маркетплейс техноло...


### Задание 2
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [ ]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [ ]:
avast_leaks = pd.DataFrame()

URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
data = {
    'emailAddresses': EMAIL
}

headers = {'Vaar-Version': '0', 'Vaar-Header-App-Product': 'hackcheck-web-avast'}

avast_hack_check_req = requests.post(URL, json=data, headers=headers)
leaks_json = avast_hack_check_req.json()

for breach in leaks_json['breaches']:
    breach_date = leaks_json['breaches'][breach]['publishDate']
    breach_date = datetime.datetime.strptime(breach_date, '%Y-%m-%dT%H:%M:%SZ').strftime('%d.%m.%Y')
    breach_site = leaks_json['breaches'][breach]['site']
    breach_desc = leaks_json['breaches'][breach]['description']
    for mail in leaks_json['summary']:
        if int(breach) in leaks_json['summary'][mail]['breaches']:
            breach_mail = mail
       
    row = {'mail': breach_mail, 'date': breach_date, 'site': breach_site, 'description': breach_desc}
    
    avast_leaks = pd.concat([avast_leaks, pd.DataFrame([row])])     
    
avast_leaks.head()

In [8]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'Vaar-Header-App-Product': 'hackcheck-web-avast','Vaar-Version': '0'}
addresses = {'emailAddresses': EMAIL}
res = requests.post(url, json=addresses, headers=headers)
df = pd.DataFrame(res.json()['breaches']).T.reset_index() 
df = df[['site', 'description', 'publishDate']]

In [9]:
df.head()

,site,description,publishDate
0,azcentral.com,"At an unconfirmed date, online Arizona newspap...",2020-01-03T00:00:00Z
1,wishbone.io,"In January 2020, the online poll website Wishb...",2020-05-28T00:00:00Z
2,myheritage.com,"In October 2017, a customer database belonging...",2017-11-04T00:00:00Z
3,canva.com,"In May 2019, graphic-design site Canva's datab...",2019-06-13T00:00:00Z
4,parapa.mail.ru,"In July and August 2016, two criminals execute...",2017-02-14T00:00:00Z


### Задание 3

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

- GROUP = 'netology'  
- TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [ ]:
# https://vk.com/dev/manuals
# https://vk.com/dev/newsfeed.search
POSTS_REQUEST = 'https://api.vk.com/method/wall.get?'
VERSION = '5.103'
GROUP = 'netology'
SLEEP = 0.33

params = {
    'access_token': '...',
    'v': VERSION,
    'domain': GROUP,
    'count': 50
}

res = requests.get(POSTS_REQUEST, params)
posts = res.json()

vk_posts = pd.DataFrame()

for post in posts['response']['items']:
    post_date = post['date']
    post_date = datetime.datetime.fromtimestamp(post_date).strftime('%d.%m.%Y')
    post_text = post['text']
    
    row = {'date': post_date, 'text': post_text}
    
    vk_posts = pd.concat([vk_posts, pd.DataFrame([row])]) 

vk_posts.head()